In [1]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from math import sqrt
import warnings

warnings.filterwarnings('ignore')

In [2]:
path_train = "../resource/PINGAN-2018-train_demo.csv"
path_test = "../resource/PINGAN-2018-test_demo.csv"

all_feature_list = ['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1','dis']

use_feature_list = [
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call_ratio_0', 'call_ratio_1', 'dis', 'ave_dri_time', 'dri_time']

In [3]:
def load_data(path_train,path_test):
    train_data = pd.read_csv(path_train)
    test_data = pd.read_csv(path_test)
    return train_data,test_data

In [4]:
#加载数据
train_data,test_data=load_data(path_train,path_test)

In [5]:
#拼接训练集和测试集进行特征工程
test_data['TERMINALNO'] = test_data['TERMINALNO']+train_data['TERMINALNO'].max()
data = pd.concat([train_data,test_data])

In [6]:
#重置index
data.reset_index(drop=True,inplace=True)

In [7]:
#时间处理
def time_datetime(value):
    format = '%Y%m%d%H%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_date(value):
    format = '%Y%m%d'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_hour(value):
    format = '%H'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)
def time_minute(value):
    format = '%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

#转换成时刻
data['time']=data['TIME'].apply(time_datetime)
data['date']=data['TIME'].apply(time_date)
data['hour']=data['TIME'].apply(time_hour)
data['minute']=data['TIME'].apply(time_minute)

In [8]:
# trip_max
feature=pd.DataFrame()
feature[['TERMINALNO','trip_max']]=pd.DataFrame(data['TRIP_ID'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','TRIP_ID']]

# lon_max lon_min lon
lonmax=pd.DataFrame()
lonmin=pd.DataFrame()
lonmax[['TERMINALNO','lon_max']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LONGITUDE']]
lonmin[['TERMINALNO','lon_min']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LONGITUDE']]
feature=pd.merge(feature,lonmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,lonmin,how='left',on='TERMINALNO')
feature['lon']=feature['lon_max']-feature['lon_min']

# lat_max lat_min lat
latmax=pd.DataFrame()
latmin=pd.DataFrame()
latmax[['TERMINALNO','lat_max']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LATITUDE']]
latmin[['TERMINALNO','lat_min']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LATITUDE']]
feature=pd.merge(feature,latmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,latmin,how='left',on='TERMINALNO')
feature['lat']=feature['lat_max']-feature['lat_min']

# heg_max heg_min heg_mean heg
hegmax=pd.DataFrame()
hegmin=pd.DataFrame()
hegmean=pd.DataFrame()
hegmax[['TERMINALNO','heg_max']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','HEIGHT']]
hegmin[['TERMINALNO','heg_min']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','HEIGHT']]
hegmean[['TERMINALNO','heg_mean']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','HEIGHT']]
feature=pd.merge(feature,hegmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmin,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmean,how='left',on='TERMINALNO')
feature['heg']=feature['heg_max']-feature['heg_min']

# volu 活动区间体积
feature['vol']=feature['lon']*feature['lat']*feature['heg']

# 速度 sp_max sp_mean
spmax=pd.DataFrame()
spmean=pd.DataFrame()
spmax[['TERMINALNO','sp_max']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','SPEED']]
spmean[['TERMINALNO','sp_mean']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','SPEED']]
feature=pd.merge(feature,spmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,spmean,how='left',on='TERMINALNO')

#callstate
call0=pd.DataFrame()
call1=pd.DataFrame()
call0[['TERMINALNO','call0']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] == 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
call1[['TERMINALNO','call1']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] > 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
feature=pd.merge(feature,call0,how='left',on='TERMINALNO')
feature=pd.merge(feature,call1,how='left',on='TERMINALNO')

feature['call0'].fillna(0,inplace=True)
feature['call1'].fillna(0,inplace=True)
feature['call_ratio_0']=feature['call0']/(feature['call0']+feature['call1'])
feature['call_ratio_1']=feature['call1']/(feature['call0']+feature['call1'])

In [9]:
from math import sqrt
# 行程
# 对每个USER 按 TIME 排序
sortdata = data.sort_values(['TERMINALNO','time']).reset_index(drop = True)
# 删除TRIP_ID后去重
del sortdata['TRIP_ID']
sortdata.drop_duplicates(inplace=True)
# 计算经纬度差
sortdata['difflat'] = sortdata.groupby(['TERMINALNO'])['LATITUDE'].diff()
sortdata['difflon'] = sortdata.groupby(['TERMINALNO'])['LONGITUDE'].diff()
# 对每个用户的第一个经纬度差置0
sortdata.fillna(0.0,inplace=True)
# 计算单个距离
sortdata['dis2'] = sortdata['difflat'] ** 2 + sortdata['difflon'] ** 2
sortdata['dis'] = sortdata['dis2'].apply(sqrt)
del sortdata['dis2']
# 计算总行程
# disdata = pd.DataFrame()
# disdata[['TERMINALNO','dis']]=sortdata['dis'].groupby(['TERMINALNO']).sum()
disdata = sortdata['dis'].groupby(sortdata['TERMINALNO']).sum().reset_index()
feature = pd.merge(feature,disdata,how='left',on='TERMINALNO')

In [10]:
# 驾驶时长 
# 1.去重
dri_time = data[['TERMINALNO','TIME','TRIP_ID']]
dri_time.drop_duplicates(subset=['TERMINALNO','TIME'],inplace=True)
# 2.按 TERMINALNO 和 time 排序
dri_time.sort_values(['TERMINALNO','TIME'],inplace=True)
dri_time['diff_time']=dri_time.groupby(['TERMINALNO'])['TIME'].diff()
dri_time.fillna(0.0,inplace=True)
# 3.时间换算
dri_time['diff_time'] = dri_time['diff_time'].apply(lambda x: x / 60)
# 4.如果时间间隔大于20分钟则按新行程处理，置0
def f(x):
    if x >= 20:
        return 0
    else:
        return x
dri_time['diff_time'] = dri_time['diff_time'].apply(f)
# 5.计算驾驶总时长
dri_t = pd.DataFrame()
dri_t[['TERMINALNO','dri_time']] = dri_time['diff_time'].groupby(dri_time['TERMINALNO']).sum().reset_index()[['TERMINALNO','diff_time']]
feature = pd.merge(feature,dri_t,how='left',on='TERMINALNO')
# 6.平均时长
feature['ave_dri_time'] = feature['dri_time'] / feature['trip_max'] 
# 7.用户单段最大驾驶时长
del dri_t,dri_time

In [11]:
# data.sort_values(by=['TIME'], inplace=True)
# data['diff_time'] = data.groupby(['TERMINALNO', 'TRIP_ID'])['TIME'].diff()
# value = {'diff_time': 0}
# data.fillna(value=value, inplace=True)
# dri_time = pd.DataFrame()
# dri_time[['TERMINALNO', 'dri_time']] = pd.DataFrame(data.groupby(['TERMINALNO'])['diff_time'].sum()).reset_index()
# dri_time['dri_time'] = dri_time['dri_time'].apply(lambda x: x / 60)
# feature = pd.merge(feature, dri_time, how='left', on='TERMINALNO')

In [12]:
# 归一化
feature['trip_max'] = feature['trip_max'].apply(
    lambda x: (x - feature['trip_max'].min()) / (feature['trip_max'].max() - feature['trip_max'].min()))
feature['lon_max'] = feature['lon_max'].apply(
    lambda x: (x - feature['lon_max'].min()) / (feature['lon_max'].max() - feature['lon_max'].min()))
feature['lon_min'] = feature['lon_min'].apply(
    lambda x: (x - feature['lon_min'].min()) / (feature['lon_min'].max() - feature['lon_min'].min()))
feature['lon'] = feature['lon'].apply(
    lambda x: (x - feature['lon'].min()) / (feature['lon'].max() - feature['lon'].min()))
feature['lat_min'] = feature['lat_min'].apply(
    lambda x: (x - feature['lat_min'].min()) / (feature['lat_min'].max() - feature['lat_min'].min()))
feature['lat_max'] = feature['lat_max'].apply(
    lambda x: (x - feature['lat_max'].min()) / (feature['lat_max'].max() - feature['lat_max'].min()))
feature['lat'] = feature['lat'].apply(
    lambda x: (x - feature['lat'].min()) / (feature['lat'].max() - feature['lat'].min()))
feature['heg_min'] = feature['heg_min'].apply(
    lambda x: (x - feature['heg_min'].min()) / (feature['heg_min'].max() - feature['heg_min'].min()))
feature['heg_max'] = feature['heg_max'].apply(
    lambda x: (x - feature['heg_max'].min()) / (feature['heg_max'].max() - feature['heg_max'].min()))
feature['heg'] = feature['heg'].apply(
    lambda x: (x - feature['heg'].min()) / (feature['heg'].max() - feature['heg'].min()))
feature['heg_mean'] = feature['heg_mean'].apply(
    lambda x: (x - feature['heg_mean'].min()) / (feature['heg_mean'].max() - feature['heg_mean'].min()))
feature['vol'] = feature['vol'].apply(
    lambda x: (x - feature['vol'].min()) / (feature['vol'].max() - feature['vol'].min()))
feature['sp_max'] = feature['sp_max'].apply(
    lambda x: (x - feature['sp_max'].min()) / (feature['sp_max'].max() - feature['sp_max'].min()))
feature['sp_mean'] = feature['sp_mean'].apply(
    lambda x: (x - feature['sp_mean'].min()) / (feature['sp_mean'].max() - feature['sp_mean'].min()))
feature['ave_dri_time'] = feature['ave_dri_time'].apply(
    lambda x: (x - feature['ave_dri_time'].min()) / (feature['ave_dri_time'].max() - feature['ave_dri_time'].min()))
feature['dri_time'] = feature['dri_time'].apply(
    lambda x: (x - feature['dri_time'].min()) / (feature['dri_time'].max() - feature['dri_time'].min()))
feature['dis'] = feature['dis'].apply(
    lambda x: (x - feature['dis'].min()) / (feature['dis'].max() - feature['dis'].min()))


In [ ]:
# 切割训练集和测试集
train = data[0:len(train_data)]
test = data[len(train_data):]
train = pd.merge(train, feature, how='left', on='TERMINALNO')
test = pd.merge(test, feature, how='left', on='TERMINALNO')
train['Y'] = train['Y'].apply(lambda x: ((x-train['Y'].min())/(train['Y'].max()-train['Y'].min())))
# 训练集和验证集划分
train_train, train_val = train_test_split(train, test_size=0.2, random_state=42)
print("train_train_shape:"+str(train_train.shape)+"  train_val_shape:"+str(train_val.shape))

# train_train['Y'] = train_train['Y'].apply(
#     lambda x: ((x - train_train['Y'].min())/(train_train['Y'].max()-train_train['Y'].min()))
# )
# train_val['Y'] = train_val['Y'].apply(
#     lambda x: ((x - train_val['Y'].min()) / (train_val['Y'].max() - train_val['Y'].min()))
# )

#模型训练
lgbmodel = lgb.LGBMRegressor(num_leaves=63, max_depth=7, n_estimators=20000,learning_rate=0.0001,n_jobs=20,random_state=2018)
lgbmodel.fit(X=train_train[use_feature_list], y=train_train['Y'],
             eval_set=(train_val[use_feature_list], train_val['Y']), early_stopping_rounds=200,verbose=False)

fea_imp = pd.Series(lgbmodel.feature_importances_,use_feature_list).sort_values(ascending=False)
print(fea_imp)

train_train_shape:(55444, 35)  train_val_shape:(13862, 35)
